In [30]:
import cv2
import numpy as np
import time
import csv

In [31]:
VIDEO_PATH = "trafico01.mp4"
FPS = 25
SHOW_VIDEO = True

# Líneas de conteo (coordenadas originales del video)
# Formato: [x1, y1, x2, y2]
LINES = [
    [480, 850, 580, 850],   # Línea 1
    [610, 850, 720, 850],   # Línea 2
    [990, 750, 1090, 750],  # Línea 3
    [1320, 900, 1470, 900], # Línea 4
    [1180, 640, 1210, 640], # Línea 5
    [1420, 730, 1500, 730], # Línea 6
    [1630, 770, 1770, 770], # Línea 7
]

# Desplazamiento vertical de cada línea (para afinar posición)
# Positivo = hacia abajo, Negativo = hacia arriba
LINE_OFFSETS_Y = [0, 0, 0, 0, -25, +25, 0]

# Escala para adaptar a video redimensionado
RESIZE_W, RESIZE_H = 960, 540
ORIG_W, ORIG_H = 1920, 1080  # resolución original estimada

In [32]:
def scale_lines(lines, orig_w, orig_h, new_w, new_h, offsets_y=None):
    sx, sy = new_w / orig_w, new_h / orig_h
    scaled = []
    offsets_y = offsets_y or [0]*len(lines)
    for i, (x1, y1, x2, y2) in enumerate(lines):
        dy = offsets_y[i] if i < len(offsets_y) else 0
        scaled.append({
            "p1": (int(x1*sx), int(y1*sy)+dy),
            "p2": (int(x2*sx), int(y2*sy)+dy),
            "count": 0,
            "cars": set()
        })
    return scaled

def line_crossed(cx, cy, line, tol=30):
    """Devuelve True si el punto (cx, cy) cruza una línea"""
    x1, y1 = line["p1"]
    x2, y2 = line["p2"]
    return (x1 <= cx <= x2) and (abs(cy - y1) <= tol)

def find_car_id(cx, cy, tracks, max_distance=40):
    """Asigna un ID al coche más cercano si existe"""
    for cid, pts in tracks.items():
        x_last, y_last, _ = pts[-1]
        if np.hypot(cx - x_last, cy - y_last) < max_distance:
            return cid
    return None

def smooth_centroid(cid, cx, cy, tracks, alpha=0.4):
    """Suaviza el movimiento del centroide (filtro exponencial)"""
    if cid not in tracks or len(tracks[cid]) < 1:
        return cx, cy
    prev_x, prev_y, _ = tracks[cid][-1]
    return int(alpha*cx + (1-alpha)*prev_x), int(alpha*cy + (1-alpha)*prev_y)

In [33]:
lines = scale_lines(LINES, ORIG_W, ORIG_H, RESIZE_W, RESIZE_H, LINE_OFFSETS_Y)

cap = cv2.VideoCapture(VIDEO_PATH)
ret, f1 = cap.read()
ret, f2 = cap.read()
if not ret:
    raise RuntimeError("No se pudo abrir el video")

car_tracks = {}
next_car_id = 0
frame_id = 0

start = time.time()

while ret:
    frame_id += 1
    f1 = cv2.resize(f1, (RESIZE_W, RESIZE_H))
    f2 = cv2.resize(f2, (RESIZE_W, RESIZE_H))

    gray1 = cv2.cvtColor(f1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(f2, cv2.COLOR_BGR2GRAY)

    diff = cv2.absdiff(gray1, gray2)
    _, thresh = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
    thresh = cv2.blur(thresh, (5,5))
    _, thresh = cv2.threshold(thresh, 40, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        if cv2.contourArea(c) < 400:
            continue
        x, y, w, h = cv2.boundingRect(c)
        cx, cy = x + w//2, y + h//2

        car_id = find_car_id(cx, cy, car_tracks)
        if car_id is None:
            car_id = next_car_id
            car_tracks[car_id] = []
            next_car_id += 1

        # suaviza el movimiento del centroide
        cx, cy = smooth_centroid(car_id, cx, cy, car_tracks)

        car_tracks[car_id].append((cx, cy, frame_id))

        # velocidad en px/s (media de últimos 5 frames)
        if len(car_tracks[car_id]) > 5:
            x1, y1, f1_id = car_tracks[car_id][-5]
            dist = np.hypot(cx - x1, cy - y1)
            time_s = (frame_id - f1_id) / FPS
            vel = dist / time_s if time_s > 0 else 0
        else:
            vel = 0

        # Dibujo del rectángulo y texto
        cv2.rectangle(f1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(f1, f"{vel:.1f} px/s", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

        # Cruce de líneas
        for i, line in enumerate(lines):
            if line_crossed(cx, cy, line):
                if car_id not in line["cars"]:
                    line["cars"].add(car_id)
                    line["count"] += 1
                cv2.line(f1, line["p1"], line["p2"], (0,0,255), 3)
            else:
                cv2.line(f1, line["p1"], line["p2"], (0,255,0), 2)
            # Mostrar conteo por línea
            cv2.putText(f1, str(line["count"]), (line["p1"][0], line["p1"][1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    if SHOW_VIDEO:
        cv2.imshow("Conteo y Velocidad (px/s)", f1)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    f1 = f2
    ret, f2 = cap.read()

cap.release()
cv2.destroyAllWindows()

In [34]:
for i, line in enumerate(lines, 1):
    print(f"Línea {i}: {line['count']} coches")

Línea 1: 3 coches
Línea 2: 2 coches
Línea 3: 7 coches
Línea 4: 6 coches
Línea 5: 4 coches
Línea 6: 4 coches
Línea 7: 4 coches
